In [1]:
from imports import *
from functions import *
from constants import *
# Active l'extension autoreload dans un notebook Jupyter
%load_ext autoreload
%autoreload 2

# Launching Driver and Website Main Page

In [3]:
driver = create_driver(website_url)

In [4]:
driver.get(website_url)
destroy_intrusive_elements(driver)
wait = WebDriverWait(driver, timeout=10)

############## CONNECTION #######################################################################################################################################
mi_cuenta = wait.until(EC.presence_of_element_located((By.XPATH, "//span[normalize-space()='Mi cuenta']")))  # Remplacer par l'élément attendu
mi_cuenta.click()
# Envoie de l'email sur le site
try:
    ingresar_email_zone = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Ingresa tu email']")))
    ingresar_email_zone.clear()
    ingresar_email_zone.send_keys(EMAIL)
    ingresar_email_zone.send_keys(Keys.ENTER)
except:
    print('NO')

# Connection

# Search Mechanic

In [5]:
SEARCH_QUERY = "portatil ASUS 1TB RTX RYZEN"

Dynamically Building the Name of the CSV File to store collected Data

In [6]:
csv_output_name = f"Exito_Search_{SEARCH_QUERY.replace(" ", "_")}.csv"

dict_results = {}
today = dt.now()
dict_results["Timestamp"] = today

csv_output_name

'Exito_Search_portatil_ASUS_1TB_RTX_RYZEN.csv'

# Launch Query

In [ ]:
driver.get(website_url)

try:
    # Attendre que la page soit complètement chargée
    

    # Try to find the search field
    try:
        search_field = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//input[@data-fs-input='true' and @aria-label='search']")
            )
        )
        print("Search field found and is clickable!")
    except Exception as e:
        print(f"❌ Error while finding search field: {e}")
        raise

    try:
        # Ensure the field is clear (use JavaScript to clear if necessary)
        driver.execute_script("arguments[0].value = '';", search_field)
        print("Field cleared using JavaScript!")
    except Exception as e:
        print(f"❌ Error while clearing field using JavaScript: {e}")
        raise

    try:
        # Optionally, use BACKSPACE to ensure the field is emptied
        current_value = search_field.get_attribute('value')
        if current_value:
            search_field.send_keys(Keys.BACKSPACE * len(current_value))
            print("Text cleared using BACKSPACE.")
        else:
            print("No text to clear with BACKSPACE.")
    except Exception as e:
        print(f"❌ Error while using BACKSPACE to clear text: {e}")
        raise

    try:
        # Send the search query
        search_field.send_keys(SEARCH_QUERY)
        search_field.send_keys(Keys.ENTER)
        print(f"Search query '{SEARCH_QUERY}' sent!")
    except Exception as e:
        print(f"❌ Error while sending search query: {e}")
        raise

except Exception as e:
    print(f"General error during search process: {str(e)}")

Page completely loaded!
Search field found and is clickable!
Field cleared using JavaScript!
No text to clear with BACKSPACE.
Search query 'portatil ASUS 1TB RTX RYZEN' sent!


# Data Collection

One Card

In [ ]:
products_listing = find_element(driver, "xpath", "//div[contains(@data-fs-product-listing-results, '')]")
cards = find_elements(products_listing, "xpath", ".//article[contains(@class, 'productCard')]")
card = cards[0]
title = find_element(card, "xpath", ".//h3[contains(@class, 'name')]").text.strip()
print(f"Titre : {title}")
link = find_element(card, "xpath", ".//a[contains(@data-testid,'product-link')]").get_attribute('href')
print(f"Link : {link}")

brand = find_element(card, "xpath", ".//h3[contains(@class, 'brand')]").text.strip()
print(f"Marque : {brand}")
actual_price = find_element(card, "xpath", ".//p[contains(@class, 'ProductPrice')]").text.strip()
print(f"Prix : {actual_price}")
try:
    last_price = find_element(card, "xpath", ".//p[contains(@class, 'promotion_price')]").text.strip()
    print(f"Ancien prix : {last_price}")
    # calculer le pourcentage de réduction
    discount = get_discount_percentage(last_price, actual_price)
    print(f"Réduction : {discount}%\n")

except NoSuchElementException or TimeoutException:
    print("Last Price not found...")

driver.get(link)
wait_page_to_load(driver)

Titre : Portatil Asus Proart P16 Oled Amd Ryzen 9 Rtx 4070 Ram 32Gb Ssd 1Tb Windows 11 Home
Link : https://www.exito.com/portatil-asus-proart-p16-oled-amd-ryzen-9-rtx-4070-ram-32gb-ssd-1tb-windows-11-home-104055765-mp/p
Marque : ASUS
Prix : $ 13.112.900
Ancien prix : $ 17.466.900
Réduction : 25%



# Product Page Scraping

In [40]:
try:
    product_section = find_element(driver, "xpath", 
                                   "//section[contains(@data-fs-product-container,'true')]")
    print(f"Product Section Found !")
except NoSuchElementException:
    print("Product Section Not Found !")
try:
    Dealer = find_element(product_section, "xpath", 
                      ".//a[contains(@href, 'seller')]").text.strip()
    print(f"Dealer: {Dealer}")
except NoSuchElementException:
    print("Dealer Not Found !")

try:
    stock_details = find_element(product_section, "xpath", 
        ".//p[contains(@data-fs-product-details-stock__qty,'true')]")
    print(f"Stock details: {stock_details.text.strip()}")
except NoSuchElementException:
    print("Dealer Not Found !")

try:
    PLU_ID = find_element(product_section, "xpath",
        ".//span[contains(@class,'product-title') and contains(@class,'specification')]")
    print(f"PLU_ID: {PLU_ID.text.strip()}")
except NoSuchElementException:
    print("PLU not found !")

try:
    compra_y_recoge = find_element(product_section, "xpath",
    ".//p[contains(@data-fs-promises-paragraph,'true') and contains(@data-fs-promises-bold,'true')]").text.strip()
    print(f"{compra_y_recoge} para compra y recoge") 
except NoSuchElementException:
    print("Compra y recoge data not found !")    

try:
    garantia = find_element(product_section, "xpath",
    ".//p[contains(@data-fs-product-aditional-info-garantia__txt,'true')]").text.strip()
    print(f"{garantia}") 
except NoSuchElementException:
    print("Guarantee data not found !")  

try:
    img_zone = find_element(product_section, "xpath",
    ".//section[contains(@data-fs-product-details-gallery,'true')]")
    print(f"Image Zone Found !") 

    try:
        images = find_elements(img_zone, "xpath",
        ".//img[contains(@src,'') and contains(@alt, '') and contains(@src,'')]")
        print(f"{len(images)} images Found !")
        list_images = []
        for i, image in enumerate(images):
            image_url = image.get_attribute("src")
            print(f"Image {i+1}: {image_url}")
            list_images.append(image_url)


    except NoSuchElementException:
     print("Images not found !")  
except NoSuchElementException:
    print("Image Zone not found !")


try:
    Products_specifications = find_element(product_section, "xpath",
                                           
    ".//section[contains(@class, 'product-specifications_fs')]")
    print(f"Products Specifications Found !") 

except Exception:
    print("Products Specifications not found !")
 

Product Section Found !
Dealer: Smartbuy_co
Stock details: ¡Últimas 1 unidades!
PLU_ID: ASUS-PLU: 104055765
No disponible para compra y recoge
Garantía del producto: 12 meses.
Image Zone Found !
10 images Found !
Image 1: https://exitocol.vtexassets.com/arquivos/ids/25918247/portatil-asus-proart-p16-oled-amd-ryzen-9-rtx-4070-ram-32gb-ssd-1tb-windows-11-home.jpg?v=638706010037170000
Image 2: https://exitocol.vtexassets.com/arquivos/ids/25918248/portatil-asus-proart-p16-oled-amd-ryzen-9-rtx-4070-ram-32gb-ssd-1tb-windows-11-home.jpg?v=638706010038730000
Image 3: https://exitocol.vtexassets.com/arquivos/ids/25918249/portatil-asus-proart-p16-oled-amd-ryzen-9-rtx-4070-ram-32gb-ssd-1tb-windows-11-home.jpg?v=638706010041400000
Image 4: https://exitocol.vtexassets.com/arquivos/ids/25918250/portatil-asus-proart-p16-oled-amd-ryzen-9-rtx-4070-ram-32gb-ssd-1tb-windows-11-home.jpg?v=638706010043270000
Image 5: https://exitocol.vtexassets.com/arquivos/ids/25918251/portatil-asus-proart-p16-oled-amd-ry

One Card

In [35]:
category = "Tecnología"
article_type_choice = "Portátiles"

main_page=driver.current_url
def go_to_article_page(driver, category, article_type):
    import time
    from selenium.webdriver.common.by import By

    try:
        # Récupère le menu des catégories
        categories_menu = driver.find_element(By.XPATH, '//*[@id="header-page"]/aside/ul/section[2]')

        # Cherche tous les div ayant l'attribut data-link-content="true"
        divs_with_data_link_content = categories_menu.find_elements(By.XPATH, './/div[@data-link-content="true"]')

        # Clique sur la catégorie principale
        for div in divs_with_data_link_content:
            if div.text.strip() == category:
                div.click()
                print(f"Catégorie '{category}' cliquée.")
                time.sleep(1)
                break

        # Menu déroulé : chercher le type d'article
        selected_menu = driver.find_element(By.XPATH, './/ul[@data-content-list="true"]')
        list_elements = selected_menu.find_elements(By.TAG_NAME, 'li')

        for li in list_elements:
            if li.text.strip() == article_type:
                try:
                    link = li.find_element(By.TAG_NAME, 'a')
                    link.click()
                    print(f"Lien cliqué vers la page '{article_type}'")
                    time.sleep(2)
                    return driver.current_url
                except:
                    print(f"Lien introuvable dans l'élément '{article_type}'")
                    return None
        print(f"Type d'article '{article_type}' non trouvé.")
        return None

    except Exception as e:
        print(f"Erreur lors de la navigation : {e}")
        return None
try:
    go_to_article_page(driver, category, article_type_choice)
except Exception as e:
    print(str(e))

Erreur lors de la navigation : Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="header-page"]/aside/ul/section[2]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF791E8EFA5+77893]
	GetHandleVerifier [0x00007FF791E8F000+77984]
	(No symbol) [0x00007FF791C591BA]
	(No symbol) [0x00007FF791CAF16D]
	(No symbol) [0x00007FF791CAF41C]
	(No symbol) [0x00007FF791D02237]
	(No symbol) [0x00007FF791CD716F]
	(No symbol) [0x00007FF791CFF07F]
	(No symbol) [0x00007FF791CD6F03]
	(No symbol) [0x00007FF791CA0328]
	(No symbol) [0x00007FF791CA1093]
	GetHandleVerifier [0x00007FF792147B6D+2931725]
	GetHandleVerifier [0x00007FF792142132+2908626]
	GetHandleVerifier [0x00007FF7921600F3+3031443]
	GetHandleVerifier [0x00007FF791EA91EA+184970]
	GetHandleVerifier [0x00007FF791EB086F+215311]
	GetHand

Getting all the Checkboxes

In [ ]:
computer_filter_menu = driver.find_element(By.XPATH, "//div[@class='accordion_fs-accordion__mi4MR']")

# Récupère tous les titres des filtres (balises h4)
filters = computer_filter_menu.find_elements(By.TAG_NAME, 'h4')
Sub_categoría = filters[1]

# desktop-store-filter-Sub-categoría-Accesorios de cámaras

# Vérifier si l'élément Sub_categoría a été trouvé
if Sub_categoría:
    # Trouver toutes les checkboxes à l'intérieur de 'Sub_categoría'
    checkboxes = Sub_categoría.find_elements(By.XPATH, '//input[@type="checkbox" and @data-fs-checkbox="true"]')
    print(f"Nombre de checkboxes trouvées : {len(checkboxes)}")
    for checkbox in checkboxes:
        print(checkbox.get_attribute("data-value"))
else:
    print("L'élément 'Sub-categoría' n'a pas été trouvé.")

In [ ]:
def cocher_checkboxes(filters_choice, checkboxes, driver):
    # Normalisation des filtres
    filters_normalized = [unidecode(item.lower()) for item in filters_choice]

    for i in range(len(checkboxes)):
        try:
            # On relocalise à chaque boucle pour éviter stale
            checkboxes_refreshed = driver.find_elements(By.CSS_SELECTOR, "input[type='checkbox'][data-value]")
            if i >= len(checkboxes_refreshed):
                break  # Au cas où le DOM change

            checkbox = checkboxes_refreshed[i]

            data_value_raw = checkbox.get_attribute("data-value")
            if data_value_raw is None:
                continue

            data_value = unidecode(data_value_raw.replace("-", " ").lower())
            matched = any(f in data_value for f in filters_normalized)

            if matched and not checkbox.is_selected():
                click_with_JS(checkbox, driver)
                print(f"[CHECK] Checkbox pour '{data_value}' cochée.")

            elif not matched and checkbox.is_selected():
                click_with_JS(checkbox, driver)
                print(f"[UNCHECK] Checkbox pour '{data_value}' décochée.")
        
        except Exception as e:
            print(f"[ERROR] Problème sur checkbox {i}: {e}")
            continue

from unidecode import unidecode


In [ ]:
queries = [
    "asus", "8 gb", "16 GB", "nvidia", "amd", "24 gb", "32 gb", "ssd",
    "ryzen 7", "500 GB", "1 TB", "i7", "gaming", 'portatiles'
]

In [ ]:
try:
    cocher_checkboxes(filters_choice=queries, checkboxes=checkboxes,driver=driver)
except Exception as e:
    print(str(e))

In [ ]:
from currencies import Currency

class CurrencyConverter:
    def __init__(self, from_currency, to_currency, exchange_rate):
        self.from_currency = Currency(from_currency)
        self.to_currency = Currency(to_currency)
        self.exchange_rate = exchange_rate

    def convert(self, amount):
        # Effectuer la conversion de l'euro vers le COP
        amount_in_target_currency = amount * self.exchange_rate
        # Retourne un float sans formatage en chaîne
        return amount_in_target_currency

# Exemple d'utilisation :
# Taux de conversion (peut être récupéré depuis une API, ici 1 EUR = 4500 COP)
exchange_rate_eur_to_cop = 4500

# Création d'un convertisseur de devise
converter_eur_cop = CurrencyConverter('EUR', 'COP', exchange_rate_eur_to_cop)


In [ ]:
# Exemple d'utilisation
min_value = converter_eur_cop.convert(300)  # Conversion de 300 EUR
max_value = converter_eur_cop.convert(500)  # Conversion de 450 EUR

In [ ]:
def set_slider_value(driver, min_value, max_value):
    # Récupérer la valeur actuelle des deux sliders avant de les changer
    initial_left_value = driver.execute_script("""
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    return leftSlider ? leftSlider.value : null;
    """)
    
    initial_right_value = driver.execute_script("""
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    return rightSlider ? rightSlider.value : null;
    """)

    print(f"Valeur initiale du slider gauche : {initial_left_value}")
    print(f"Valeur initiale du slider droit : {initial_right_value}")

    # Changer la valeur des sliders via JavaScript
    script = """
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    
    if (leftSlider) {
        leftSlider.value = arguments[0]; // Définir la nouvelle valeur du slider gauche
        leftSlider.dispatchEvent(new Event('input', { bubbles: true })); // Simuler l'événement 'input'
        leftSlider.dispatchEvent(new Event('change', { bubbles: true })); // Simuler l'événement 'change'
    }

    if (rightSlider) {
        rightSlider.value = arguments[1]; // Définir la nouvelle valeur du slider droit
        rightSlider.dispatchEvent(new Event('input', { bubbles: true })); // Simuler l'événement 'input'
        rightSlider.dispatchEvent(new Event('change', { bubbles: true })); // Simuler l'événement 'change'
    }
    """
    # Exécuter le script pour changer les valeurs des deux sliders
    driver.execute_script(script, min_value, max_value)

    # Récupérer la nouvelle valeur des deux sliders après modification
    final_left_value = driver.execute_script("""
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    return leftSlider ? leftSlider.value : null;
    """)

    final_right_value = driver.execute_script("""
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    return rightSlider ? rightSlider.value : null;
    """)

    print(f"Valeur après modification du slider gauche : {final_left_value}")
    print(f"Valeur après modification du slider droit : {final_right_value}")

    # Comparer les valeurs avant et après
    if initial_left_value != final_left_value:
        print(f"Le slider gauche a été mis à jour avec succès. Nouvelle valeur : {final_left_value}")
    else:
        print(f"Aucune modification n'a été effectuée sur le slider gauche. Valeur actuelle : {final_left_value}")

    if initial_right_value != final_right_value:
        print(f"Le slider droit a été mis à jour avec succès. Nouvelle valeur : {final_right_value}")
    else:
        print(f"Aucune modification n'a été effectuée sur le slider droit. Valeur actuelle : {final_right_value}")


In [ ]:
set_slider_value(driver, min_value, max_value)

In [ ]:
aplicar_filtros = driver.find_element(By.XPATH, "//button[normalize-space()='Aplicar filtros']")
aplicar_filtros.click()